In [15]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [16]:
def imdb_data(pageno):
    #url = "https://www.imdb.com/search/title/?release_date=2020&sort=num_votes,desc&page=1"
    url = "https://www.imdb.com/search/title/?release_date=2020-01-01,2020-12-31&sort=num_votes,desc&start="+str(50*(pageno-1)+1)+"&ref_=adv_nxt"
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser') 
    complete = [] 
    for d in soup.find_all('div', attrs={'class' : "lister-item mode-advanced"}):
        head = d.find('h3', attrs={'class':"lister-item-header"})#.replace('\n','')
        movie_name = head.find('a')

        imdb_div = d.find('div', attrs={'class':"inline-block ratings-imdb-rating"})
        imdb_rating = imdb_div.find('strong')
        
        desc_div = d.find('div', attrs={'class':"lister-item-content"})
        desc = desc_div.find_all('p', attrs={'class':"text-muted"})[1:2]
        
        votes_p = d.find('p', attrs={'class':"sort-num_votes-visible"})
        votes = votes_p.find('span', attrs={'name':'nv'})
        
        direc_ = d.find('p', attrs={'class':""})
        direc=[]
        
        check = direc_.find('span')
        #print(check)
        
        for name in direc_.find_all(['a', 'span']):
            direc.append(name)

        # direc contains both directors and stars (if available) separated by a span
        direc_final=[]
        stars_final=[]
        if check is not None:
            for el in direc:
                direc_final.append(el)
                if el == check :
                    break
            
            for el in direc:
                if(el not in direc_final):
                    stars_final.append(el)
            
            direc_final.pop()
            
        else:
            stars_final = direc
            

        meta_score = d.find('div', attrs={'class':"inline-block ratings-metascore"})
        
        genre = d.find('span', attrs={'class':"genre"})
        
        duration = d.find('span', attrs={'class':"runtime"})
            
        sub = []
        
        # dealing with missing data                
        if movie_name is not None: 
            sub.append(movie_name.text)
        else:
            sub.append('NaN')
        if imdb_rating is not None: 
            sub.append(imdb_rating.text)
        else:
            sub.append('NaN')
        if desc is not None: 
            sub.append(desc[0].text.strip())
        else:
            sub.append('NaN')
        if votes is not None: 
            sub.append(votes.text)
        else:
            sub.append('NaN')
        
        if direc_final != []: 
            sub.append(direc_final) 
        else:
            sub.append('NaN')
        
        if stars_final != []:
            sub.append(stars_final)
        else:
            sub.append('NaN')

        if meta_score is not None: 
            sub.append(meta_score.text.replace("\n","")[:3]) 
        else:
            sub.append('NaN')
        if genre is not None: 
            sub.append(genre.text.strip())
        else:
            sub.append('NaN')
        if duration is not None: 
            sub.append(duration.text)
        else:
            sub.append('NaN')
        
        complete.append(sub)
        
    return complete   

In [17]:
# imdb_data(1)
movies_df = pd.DataFrame(imdb_data(1), columns = ['Movie Name', 'IMDB Rating', 'Description','Votes', 'Directors','Stars', 'Metascore', 'Genre', 'Duration'])
movies_df.head(10)

,Movie Name,IMDB Rating,Description,Votes,Directors,Stars,Metascore,Genre,Duration
0,Tenet,7.3,"Armed with only one word, Tenet, and fighting ...","496,785",[[Christopher Nolan]],"[[John David Washington], [Robert Pattinson], ...",69,"Action, Sci-Fi, Thriller",150 min
1,The Queen's Gambit,8.6,"Orphaned at the tender age of nine, prodigious...","439,889",NaN,"[[Anya Taylor-Joy], [Chloe Pirrie], [Bill Camp...",NaN,Drama,60 min
2,Soul,8.0,"After landing the gig of a lifetime, a New Yor...","326,044","[[Pete Docter], [Kemp Powers]]","[[Jamie Foxx], [Tina Fey], [Graham Norton], [R...",83,"Animation, Adventure, Comedy",100 min
3,Wonder Woman 1984,5.4,"Diana must contend with a work colleague, and ...","265,639",[[Patty Jenkins]],"[[Gal Gadot], [Chris Pine], [Kristen Wiig], [P...",60,"Action, Adventure, Fantasy",151 min
4,Birds of Prey and the Fantabulous Emancipation...,6.0,"After splitting with the Joker, Harley Quinn j...","237,841",[[Cathy Yan]],"[[Margot Robbie], [Rosie Perez], [Mary Elizabe...",60,"Action, Adventure, Comedy",109 min
5,The Invisible Man,7.1,When Cecilia's abusive ex takes his own life a...,"227,483",[[Leigh Whannell]],"[[Elisabeth Moss], [Oliver Jackson-Cohen], [Ha...",72,"Drama, Horror, Mystery",124 min
6,A Quiet Place Part II,7.2,"Following the events at home, the Abbott famil...","225,035",[[John Krasinski]],"[[Emily Blunt], [Millicent Simmonds], [Cillian...",71,"Drama, Horror, Sci-Fi",97 min
7,Ted Lasso,8.8,American college football coach Ted Lasso head...,"207,869",NaN,"[[Jason Sudeikis], [Brett Goldstein], [Brendan...",NaN,"Comedy, Drama, Sport",30 min
8,Extraction,6.7,"Tyler Rake, a fearless black market mercenary,...","202,128",[[Sam Hargrave]],"[[Chris Hemsworth], [Bryon Lerum], [Ryder Leru...",56,"Action, Thriller",116 min
9,The Trial of the Chicago 7,7.7,The story of 7 people on trial stemming from v...,"176,763",[[Aaron Sorkin]],"[[Eddie Redmayne], [Alex Sharp], [Sacha Baron ...",76,"Drama, History, Thriller",129 min


In [18]:
result = []
number_of_pages = 20 
for i in range(1, number_of_pages + 1):
    result.append(imdb_data(i))

In [19]:
list2 = []
def flatten(list1):
    for sublist1 in list1:
        for sublist2 in sublist1:
            list2.append(sublist2)
    return list2

In [20]:
movies_df = pd.DataFrame(flatten(result), columns = ['Movie Name', 'IMDB Rating', 'Description','Votes', 'Directors','Stars', 'Metascore', 'Genre', 'Duration'])

In [21]:
movies_df.shape

(1000, 9)

In [22]:
movies_df.head()

,Movie Name,IMDB Rating,Description,Votes,Directors,Stars,Metascore,Genre,Duration
0,Tenet,7.3,"Armed with only one word, Tenet, and fighting ...","496,785",[[Christopher Nolan]],"[[John David Washington], [Robert Pattinson], ...",69,"Action, Sci-Fi, Thriller",150 min
1,The Queen's Gambit,8.6,"Orphaned at the tender age of nine, prodigious...","439,889",NaN,"[[Anya Taylor-Joy], [Chloe Pirrie], [Bill Camp...",NaN,Drama,60 min
2,Soul,8.0,"After landing the gig of a lifetime, a New Yor...","326,044","[[Pete Docter], [Kemp Powers]]","[[Jamie Foxx], [Tina Fey], [Graham Norton], [R...",83,"Animation, Adventure, Comedy",100 min
3,Wonder Woman 1984,5.4,"Diana must contend with a work colleague, and ...","265,639",[[Patty Jenkins]],"[[Gal Gadot], [Chris Pine], [Kristen Wiig], [P...",60,"Action, Adventure, Fantasy",151 min
4,Birds of Prey and the Fantabulous Emancipation...,6.0,"After splitting with the Joker, Harley Quinn j...","237,841",[[Cathy Yan]],"[[Margot Robbie], [Rosie Perez], [Mary Elizabe...",60,"Action, Adventure, Comedy",109 min
